In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import geopandas as gpd

In [2]:
# Read highway exits data
highway_exits = gpd.read_file('DMTI_2019_CMCS_HighwayExitsPoint\DMTI_2019_CMCS_HighwayExitsPoint.shp')
highway_exits

,UID,RDS_UID,RDS_ID,EXIT_NUM,EXIT_DIR,MUNICIPAL,PROV,LONGITUDE,LATITUDE,ELEVATION,ACCURACY,ACQ_TECH,geometry
0,1CAEF15B-FB78-4D49-8FF7-BF885BF2C219,B2F3FD39-5C72-404F-8064-A751B824CDFC,5950409,COLLECTORS TO EXPRESS,None,TORONTO,ON,-79.374638,43.765866,163.524,30,6,POINT (-79.37464 43.76587)
1,B60EB78C-81C1-44B8-A424-CB73458F93EC,B1CB778B-C3B9-4BCF-B232-B285C58186E4,5950410,EXPRESS TO COLLECTORS,None,TORONTO,ON,-79.382197,43.764164,173.032,30,6,POINT (-79.38220 43.76416)
2,49E41B76-834A-44F2-8FA2-308106592CDA,85B11ADC-183E-4382-92DA-F0492232E4D2,5950411,375,N,TORONTO,ON,-79.341759,43.766381,162.498,3,5,POINT (-79.34176 43.76638)
3,1ADA2940-9F99-43E1-AF8A-31D81AAA19FC,75C52BE3-B86B-4E38-B360-0BA66A7284A6,5950412,EXPRESS TO COLLECTORS,None,TORONTO,ON,-79.301776,43.770823,180.003,30,6,POINT (-79.30178 43.77082)
4,502B9024-05AB-4CC7-9523-148F28877325,EB03A95B-E7C1-4D25-B827-515428D7068B,5950413,COLLECTORS TO EXPRESS,None,TORONTO,ON,-79.274200,43.777027,170.000,30,6,POINT (-79.27420 43.77703)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2800834D-CDCD-4749-9778-83071CF0453D,CD341EA6-A61B-45DD-B735-F7D50C5DFBDB,9745779,1,None,HALIFAX,NS,-63.642336,44.647241,87.893,10,5,POINT (-63.64234 44.64724)
3649,7AA6696E-CD09-4847-9F75-9C203E1DC94E,CD6B6A1C-9A9F-4AA8-A749-F18117ECA1C4,9898914,169,None,GRAVENHURST,ON,-79.365586,44.908546,252.691,10,3,POINT (-79.36559 44.90855)
3650,208836FE-1591-4C07-87A9-9A65142C339D,4BCA69E6-63B5-41CF-9650-0546CEE3178C,9898916,169,None,GRAVENHURST,ON,-79.361726,44.907361,249.993,10,3,POINT (-79.36173 44.90736)
3651,62FFC19B-0C1D-4455-8100-C7BEB5F71DB6,5AA16444-0513-4EFA-9F5C-E572A7459B4F,10109656,401,None,AJAX,ON,-79.047431,43.847561,89.320,10,5,POINT (-79.04743 43.84756)


In [9]:
highway_exits.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
highway_exits.to_crs(epsg=3348, inplace=True)

In [4]:
# Read input dataframe
df = pd.read_pickle(r'.\df_persons_wsltur_with_entropy_socio_eco.pkl')

In [8]:
df.geometry

0       POINT (7227245.124 931734.887)
1       POINT (7218315.285 931469.951)
2       POINT (7224995.358 929244.907)
3       POINT (7223229.239 929613.243)
4       POINT (7227245.124 931734.887)
                     ...              
2890    POINT (7221073.295 926424.748)
2891    POINT (7220408.755 929228.035)
2892    POINT (7227854.772 933701.215)
2893    POINT (7260452.412 962703.505)
2894    POINT (7218315.285 931469.951)
Name: geometry, Length: 2895, dtype: geometry

In [11]:
def calculate_nearest_centroid(row):
    distance_to_nearest = highway_exits['geometry'].distance(row.geometry).idxmin()
    return distance_to_nearest

df['distance_highway_exit'] = df.apply(calculate_nearest_centroid, axis=1)
df['distance_highway_exit']

0        458
1         60
2        458
3        458
4        458
        ... 
2890    1140
2891    1140
2892    1583
2893    1058
2894      60
Name: distance_highway_exit, Length: 2895, dtype: int64

In [12]:
df['distance_highway_exit'].describe()

count    2895.000000
mean     1255.208981
std      1084.790070
min         4.000000
25%       458.000000
50%       968.000000
75%      2207.000000
max      3652.000000
Name: distance_highway_exit, dtype: float64

In [13]:
# Save to pickle
df.to_pickle(r'.\df_persons_wsltur_with_entropy_socio_eco_highway_exit.pkl')